# localBizdat 
### get data about local businesses (i.e. hotels, restaurants) surrounding the brewery

In [1]:
#import dependencies
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
Base = automap_base()
import requests
from config import gkey
import json
import numpy as np

In [2]:
#create engine and import data from sqlite database
engine = create_engine('sqlite:///../static/data/bredat.sqlite', echo=False)
Base.metadata.create_all(engine)

In [3]:
Base.prepare(engine, reflect=True)

In [4]:
session = Session(engine)

In [5]:
#create dataframe
brew_df = pd.read_sql_query("SELECT id, latitude, longitude FROM breweries", con=engine)
brew_df

,id,latitude,longitude
0,5016,36.0620103554184,-79.8436043406271
1,5033,36.1327196,-81.677229
2,5035,35.4368388,-82.530580304217
3,5067,35.9915653,-78.90481567784
4,5090,34.9939752576824,-78.9220969691311
...,...,...,...
265,5273,35.59410425,-82.5553147520814
266,5283,35.3206001746298,-82.4578969903109
267,5293,35.0889040369639,-84.0352081706441
268,5304,35.9774896,-78.5100617


In [6]:
#set base url for api call
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [7]:
#create columns for new data from api call
brew_df['hotel_name']=''
brew_df['hotel_address']=''

In [8]:
#hit google places api

for index, row in brew_df.iterrows():
    loc = f"{str(row.latitude)},{str(row.longitude)}"
    params = {
        "location": loc,
        "type": "lodging",
        "rankby": "distance",
        "key": gkey,
    }
    
    response = requests.get(base_url, params=params)
    results = response.json()["results"]
    
    try:
        brew_df.at[index, 'hotel_name'] = results[0]["name"]
        brew_df.at[index, 'hotel_address'] = results[0]["vicinity"]

    except:
        pass

In [9]:
brew_df

,id,latitude,longitude,hotel_name,hotel_address
0,5016,36.0620103554184,-79.8436043406271,The Dailey Renewal Retreat Bed and Breakfast,"808 Northridge Street, Greensboro"
1,5033,36.1327196,-81.677229,Blowing Rock Brewing Company,"152 Sunset Drive, Blowing Rock"
2,5035,35.4368388,-82.530580304217,Fairfield Inn & Suites by Marriott Asheville A...,"155 Underwood Road, Fletcher"
3,5067,35.9915653,-78.90481567784,Aloft Durham Downtown,"345 Blackwell Street, Durham"
4,5090,34.9939752576824,-78.9220969691311,Town & Country Estates,"3931 Legion Road, Hope Mills"
...,...,...,...,...,...
265,5273,35.59410425,-82.5553147520814,Downtown Inn & Suites,"120 Patton Avenue, Asheville"
266,5283,35.3206001746298,-82.4578969903109,The Charleston Inn,"755 North Main Street, Hendersonville"
267,5293,35.0889040369639,-84.0352081706441,Bear Hut Cabin,"430 Butterball Blvd, Murphy"
268,5304,35.9774896,-78.5100617,Wake Forest Masonic Lodge #282,"220 Wait Avenue, Wake Forest"


In [10]:
# add data to sqlite database
brew_df.to_sql('hotels', con=engine)

In [11]:
#test sqlite data
engine.execute("SELECT * FROM hotels").fetchall()[0]

(0, 5016, '36.0620103554184', '-79.8436043406271', 'The Dailey Renewal Retreat Bed and Breakfast', '808 Northridge Street, Greensboro')